To run the Pipeline (all steps from Sensor events to Activity events) : Please execute following code

In [2]:
import general.utils
data=general.utils.loadState('temp')
import pandas as pd
target=pd.DataFrame(data['meta_targets'])
feat=pd.DataFrame(data['meta_features'])
print(feat.describe())
print(target.describe())

In [3]:
X

NameError: name 'X' is not defined

In [4]:
import pandas as pd
target=pd.DataFrame(data['meta_targets'])
feat=pd.DataFrame(data['meta_features'])

# feat['weekday']=feat['time'].dt.dayofweek
# feat['month']=feat['time'].dt.month

In [10]:
target['f1'].quantile(.1)

0.48

In [ ]:
preprocessor.transform(X).shape

In [ ]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV,cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import SplineTransformer



# y=target.loc[target['method']=='FixedSlidingWindow'][['size','shift']]
# X=feat.loc[target['method']=='FixedSlidingWindow']

y=target.drop(['accuracy','precision','recall','f1'],axis=1)
X=feat

# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# y['method']=le.fit_transform(y['method'])

In [ ]:
y

In [ ]:
from feature_extraction.features import TimeSpliter,periodic_spline_transformer
numeric_features = feat.columns.drop(['time'])
numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
            transformers=[
                ("num", numeric_transformer, numeric_features),
#                 ('date',DateEncoder(),['time']),
                ('date',make_pipeline(TimeSpliter(),ColumnTransformer(
                    transformers=[
                        ("weekday", periodic_spline_transformer(7,n_splines=3), ["time_dayofweek"]),
                        ("month", periodic_spline_transformer(12,n_splines=6), ["time_month"]),
                    ]
                )),['time']),
            ]
        )

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
pipeline=make_pipeline(preprocessor, 
#                        HistGradientBoostingRegressor()
                       MultiOutputRegressor(SVC())
                      )
# pipeline.fit(X_train,y_train['size'])

In [ ]:
def evaluate(model, X, y, cv=None):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )


In [ ]:

evaluate(pipeline,X,y)

In [ ]:
pipeline.score(X_test,y_test['size'])

In [ ]:
pipeline.fit(X,y)
ty=pipeline.predict(X)
for i,y2 in enumerate(ty):
#     print((y.iloc[i]['size']-y2)*100//y.iloc[i]['size'])
    print(y2)
    
    

In [ ]:
x_new=preprocessor.fit_transform(X)
x_new.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(x_new.shape[1],))
layer1 = layers.Dense(100, activation='relu')(inputs)
layer2 = layers.Dense(200, activation='relu')(layer1)
layer3 = layers.Dense(100, activation='relu')(layer2)
classifier = layers.Dense(1, activation='softmax',name='method')(layer3)
regressions = [layers.Dense(1, activation='linear',name=x)(layer3) for x in y.columns.drop('method')]

mdl = keras.Model(inputs=inputs, outputs=[classifier, *regressions])

mdl.compile(loss=['categorical_crossentropy',*(['mse']*len(regressions))], optimizer='adam', metrics=['accuracy'])
from tensorflow.keras.utils import plot_model
plot_model(mdl)

In [ ]:
['a',*(['mse']*10)]

In [ ]:
mdl.summary()

In [ ]:
mdl.fit(x_new,m.transform(y))

In [ ]:
y

In [ ]:
y_pr=mdl.predict(x_new)
m.inverse_transform(y_pr)

In [ ]:
y_pr

In [ ]:


        
m=MyTargetTransformer()
m.fit(y)        

In [ ]:
y2=m.transform(y)
# d={c:m.trans[c].inverse_transform(y2[i]).reshape(-1) for i,c in enumerate(m.trans)}
# print(pd.DataFrame(d))
# for i,c in enumerate(m.trans):
#     print(c,i,y_pr[i])
#     print(m.trans[c].inverse_transform(y2[i]).reshape(-1))
# #     d= {c:self.trans[c].inverse_transform(y[i]) }
m.inverse_transform(m.transform(y))

In [ ]:
pd.DataFrame(np.hstack(y2))